Import des libraries

In [12]:
from pandas import DataFrame
from pandas import read_csv
import bigml.api
from bigml.api import BigML
import kaggle

Modification des features de cs-train

In [ ]:
df = read_csv('cs-training.csv', header=0, index_col=0)
df.fillna(0, inplace=True)
df['NumberOfTimeTotalDelay'] = df['NumberOfTime30-59DaysPastDueNotWorse']
                                + df['NumberOfTimes90DaysLate']
                                + df['NumberOfTime60-89DaysPastDueNotWorse']
df['isOld'] =df['age']>70
df.to_csv('train_full.csv')

Modification des features de cs_test

In [ ]:
df = read_csv('cs-test.csv', header=0, index_col=0)
df.fillna(0, inplace=True)
df['NumberOfTimeTotalDelay'] = df['NumberOfTime30-59DaysPastDueNotWorse']
                                + df['NumberOfTimes90DaysLate']
                                + df['NumberOfTime60-89DaysPastDueNotWorse']
df['isOld'] =df['age']>70
df.to_csv('test_full.csv')

Identification du projet sur BigML

In [14]:
api = BigML(project='project/5d94a344eba31d460c000235')

Création des datasets avec les cs-train et cs-test modifiés

In [ ]:
source_train = api.create_source('train_full.csv')
origin_dataset = api.create_dataset(source_train)
source_test = api.create_source('test_full.csv')
test_dataset = api.create_dataset(source_test)

Split du train dataset

In [ ]:
train_dataset = api.create_dataset(
    origin_dataset, {"name": "Training_split_Jupyter",
                     "sample_rate": 0.8, "seed": "my seed"})
val_dataset = api.create_dataset(
    origin_dataset, {"name": "Validation_split_Jupyter",
                     "sample_rate": 0.8, "seed": "my seed",
                     "out_of_bag": True})

Création du modèle avec le split de train dataset

In [ ]:
ensemble = api.create_ensemble(train_dataset,{"name": "ensemble_Jupyter",
     "objective_field": "SeriousDlqin2yrs"})

Evaluation du modèle avec val dataset

In [ ]:
evaluation = api.create_evaluation(ensemble, val_dataset)

Création du Batch Prediction, avec le test set modifié

In [ ]:
batch_prediction = api.create_batch_prediction(ensemble, test_dataset, {
    "name": "ensemble_prediction_Jupyter", "all_fields": True,
    "header": True,
    "confidence": True,
    "probabilities": True})
api.ok(batch_prediction)

Export du batch prediction en csv

In [ ]:
api.download_batch_prediction(batch_prediction,
    filename='ensemble_prediction_Jupyter.csv')

Modification du fichier csv à envoyer à Kaggle

In [ ]:
df = read_csv('ensemble_prediction_Jupyter.csv', header=0, index_col=0)
df2 =df [['1 probability']]
df2.index.names = ['Id']
df2.rename(columns={"1 probability": "Probability"},inplace=True)

In [ ]:
df2.to_csv('prediction_kaggle.csv')

Soumission à Kaggle

In [ ]:
submission_file = "prediction_kaggle.csv"
kaggle.api.competition_submit(submission_file, "BigML ensemble", "GiveMeSomeCredit")

Afficher l'évaluation du modèle

In [18]:
evaluation = api.get_evaluation('evaluation/5dacd5997811dd0137000513')
api.pprint(evaluation['object']['result'])

{   'class_names': ['0', '1'],
    'mode': {   'accuracy': 0.93283,
                'average_area_under_pr_curve': 0,
                'average_area_under_roc_curve': 0,
                'average_balanced_accuracy': 0.5,
                'average_f_measure': 0.48262,
                'average_kendalls_tau_b': 0,
                'average_ks_statistic': 0,
                'average_max_phi': 0,
                'average_phi': 0,
                'average_precision': 0.46642,
                'average_recall': 0.5,
                'average_spearmans_rho': 0,
                'confusion_matrix': [[27985, 0], [2015, 0]],
                'per_class_statistics': [   {   'accuracy': 0.93283,
                                                'balanced_accuracy': 0.5,
                                                'class_name': '0',
                                                'f_measure': 0.96525,
                                                'phi_coefficient': 0,
                                   

                                                                   [   0.99077,
                                                                       0.9965,
                                                                       0.39],
                                                                   [   0.99143,
                                                                       0.99686,
                                                                       0.38],
                                                                   [   0.99307,
                                                                       0.99743,
                                                                       0.36],
                                                                   [   0.99377,
                                                                       0.99761,
                                                                       0.35],
                                                 

                                                                     [   0.2021,
                                                                         0.01787,
                                                                         0.99283],
                                                                     [   0.21637,
                                                                         0.02084,
                                                                         0.99247],
                                                                     [   0.239,
                                                                         0.02481,
                                                                         0.9918],
                                                                     [   0.25043,
                                                                         0.02581,
                                                                         0.99146],
                

                                                                                         [   [   27682,
                                                                                                 1610,
                                                                                                 405,
                                                                                                 303],
                                                                                             0.5323],
                                                                                         [   [   27649,
                                                                                                 1590,
                                                                                                 425,
                                                                                                 336],
                                                                          

                                                                                         [   [   7120,
                                                                                                 50,
                                                                                                 1965,
                                                                                                 20865],
                                                                                             0.9918],
                                                                                         [   [   6449,
                                                                                                 42,
                                                                                                 1973,
                                                                                                 21536],
                                                                          

...

                                                                       0.49441],
                                                                   [   0.0213,
                                                                       0.18263,
                                                                       0.48686],
                                                                   [   0.02193,
                                                                       0.18859,
                                                                       0.48208],
                                                                   [   0.02237,
                                                                       0.19355,
                                                                       0.48],
                                                                   [   0.023,
                                                                       0.19801,
                                          

                                                                     [   0.01877,
                                                                         0.00825,
                                                                         0.51],
                                                                     [   0.0193,
                                                                         0.00854,
                                                                         0.50588],
                                                                     [   0.02007,
                                                                         0.0089,
                                                                         0.5],
                                                                     [   0.02063,
                                                                         0.00929,
                                                                         0.49441],
                     

                                                                                                 622],
                                                                                             0.09338],
                                                                                         [   [   1378,
                                                                                                 4008,
                                                                                                 23977,
                                                                                                 637],
                                                                                             0.09667],
                                                                                         [   [   1355,
                                                                                                 3874,
                                                                        

                                                                 [   0.56079,
                                                                     0.34077,
                                                                     0.1679],
                                                                 [   0.56774,
                                                                     0.33687,
                                                                     0.16237],
                                                                 [   0.57519,
                                                                     0.33429,
                                                                     0.15872],
                                                                 [   0.58114,
                                                                     0.32967,
                                                                     0.15372],
                                                             

                                                                  [   0.03974,
                                                                      0.42531,
                                                                      0.28212],
                                                                  [   0.04188,
                                                                      0.43623,
                                                                      0.27099],
                                                                  [   0.04199,
                                                                      0.43821,
                                                                      0.27],
                                                                  [   0.04613,
                                                                      0.45161,
                                                                      0.26],
                                                      